In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

try:
    from google.colab import drive
    drive.mount('/content/drive')
    file_path = '/content/drive/My Drive/titanic.csv'

except ImportError:
    file_path = 'datasets/titanic.csv'
    pass

In [2]:
df = pd.read_csv(file_path)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,"892,0,3,""Kelly, Mr. James"",male,34.5,0,0,33091...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"893,1,3,""Wilkes, Mrs. James (Ellen Needs)"",fem...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"894,0,2,""Myles, Mr. Thomas Francis"",male,62,0,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"895,0,3,""Wirz, Mr. Albert"",male,27,0,0,315154,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"896,1,3,""Hirvonen, Mrs. Alexander (Helga E Lin...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df.shape

(418, 12)

In [37]:
print("\nТипи даних стовпців:\n", df.dtypes)


Типи даних стовпців:
 PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [3]:
print("Пропущені значення в кожному стовпці:\n", df.isnull().sum())

df = df.fillna(df.mean(numeric_only=True))

Пропущені значення в кожному стовпці:
 PassengerId      0
Survived       418
Pclass         418
Name           418
Sex            418
Age            418
SibSp          418
Parch          418
Ticket         418
Fare           418
Cabin          418
Embarked       418
dtype: int64


In [39]:
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,0
SibSp,0
Parch,0
Ticket,0
Fare,0


In [40]:
print(f"Кількість дублікатів у датасеті: {df.duplicated().sum()}")

Кількість дублікатів у датасеті: 0


In [4]:
df.drop('Cabin', axis=1, inplace=True)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,"892,0,3,""Kelly, Mr. James"",male,34.5,0,0,33091...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"893,1,3,""Wilkes, Mrs. James (Ellen Needs)"",fem...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"894,0,2,""Myles, Mr. Thomas Francis"",male,62,0,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"895,0,3,""Wirz, Mr. Albert"",male,27,0,0,315154,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"896,1,3,""Hirvonen, Mrs. Alexander (Helga E Lin...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1}).fillna(0).astype(int)
df.dtypes

,0
PassengerId,int64
Survived,int64
Pclass,int64
Name,object
Sex,int64
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64


In [5]:

features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
X = df[features]
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Розмір тренувальної вибірки: {X_train.shape[0]}')
print(f'Розмір тестової вибірки: {X_test.shape[0]}')

Розмір тренувальної вибірки: 334
Розмір тестової вибірки: 84


In [6]:
#Будуємо три моделі класифікації: Логістична регресія, Дерево рішень,
# Випадковий ліс (Random Forest).

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier()
}

param_grids = {
    'Logistic Regression': {'C': [0.01, 0.1, 1, 10, 100]},
    'Decision Tree': {'max_depth': [3, 5, 7, None], 'min_samples_split': [2, 5, 10]},
    'Random Forest': {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7, None]}
}

best_models = {}
for name, model in models.items():
    print(f"Налаштування параметрів для {name}...")
    grid_search = GridSearchCV(model, param_grids[name], cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_models[name] = grid_search.best_estimator_
    print(f"Найкращі параметри для {name}: {grid_search.best_params_}")


Налаштування параметрів для Logistic Regression...


e:\source\projects\ML-labs\.venv\Lib\site-packages\sklearn\utils\_array_api.py:390: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


ValueError: Input y contains NaN.

In [46]:
for name, model in best_models.items():
    print(f"\nОцінка моделі: {name}")
    y_pred = model.predict(X_test)
    print(f"Точність (Accuracy): {accuracy_score(y_test, y_pred):.2f}")
    print("Матриця помилок (Confusion Matrix):")
    print(confusion_matrix(y_test, y_pred))
    print("Звіт про класифікацію (Classification Report):")
    print(classification_report(y_test, y_pred))


Оцінка моделі: Logistic Regression
Точність (Accuracy): 1.00
Матриця помилок (Confusion Matrix):
[[50  0]
 [ 0 34]]
Звіт про класифікацію (Classification Report):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        34

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84


Оцінка моделі: Decision Tree
Точність (Accuracy): 1.00
Матриця помилок (Confusion Matrix):
[[50  0]
 [ 0 34]]
Звіт про класифікацію (Classification Report):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        34

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84


Оцінка моделі: Random F

In [47]:
#Робимо прогноз для 10 випадків.
best_model_name = max(best_models, key=lambda name: accuracy_score(y_test, best_models[name].predict(X_test)))
best_model = best_models[best_model_name]
print(f"\nНайкраща модель: {best_model_name}")
predictions = best_model.predict(X_test[:10])
print("Прогноз для перших 10 випадків:", predictions)


Найкраща модель: Logistic Regression
Прогноз для перших 10 випадків: [0 1 0 0 1 0 1 0 1 0]


**Висновок:**Було побудовано три моделі класифікації: логістична регресія, дерево рішень і випадковий ліс (Random Forest). Логістична регресія була обрана найкращою завдяки своїй простоті, інтерпретованості, меншій схильності до перенавчання та подібній продуктивності в порівнянні зі складнішими моделями.